Lucas-Kanade Algoritması (Sparse Optical Flow)

In [270]:
import cv2 as cv
import numpy as np

In [271]:
# Göreceli yol ile video dosyasına erişim
#video_path = "../04_VIDEO_ANALYSIS_BACKGROUND_SUBTRACTION_AND_ROI_EXTRACTION_OF_THE_FOREGROUND/pedestrian.mp4"
video_path = "slow_traffic_small.mp4"
# VideoCapture nesnesi ile video dosyasını aç
cap = cv.VideoCapture(video_path)

In [272]:
ret, frame1 = cap.read() # İlk kareyi oku

prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY) # İlk kareyi gri tonlamaya çevir

In [273]:
# Lucas-Kanade Algoritması için parametreler
lk_params = dict(winSize = (15,15), # Pencere boyutu
                 maxLevel = 2,      # Piramit seviyeleri
                 criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)) # Durma kriterleri

# Shi-Tomasi köşe algılama parametreleri
feature_params = dict(maxCorners   = 100, # Maksimum köşe sayısı
                      qualityLevel = 0.3, # Minimum kalite seviyesi
                      minDistance  = 7,   # Köşeler arası minimum mesafe
                      blockSize    = 7)   # Pencere boyutu

In [274]:
def lucas_kanade_opt_flow(prvs):
    # ilk karedeki özellik noktalarını algıla
    p0 = cv.goodFeaturesToTrack(prvs,             # 'prvs' gri tonlamalı görüntüdeki köşeleri algıla
                                mask = None,      # Maske uygulanmaz, tüm görüntü üzerinde işlem yapılır.  
                                **feature_params) # 'feature_params' sözlük parametreleri unpacking ile geçirilir.
    
    # Noktaları çizmek için rastgele renkler oluştur
    color = np.random.randint(0, 255, (100, 3)) # 100 farklı nokta için rastgele renk oluşturulur.

    # Noktaları çizmek için mask image oluştur
    mask = np.zeros_like(frame1) # Mask, ilk kare ile aynı boyutlarda sıfırlarla dolu bir görüntü oluşturur.

    while 1: # Sonsuz döngü
        ret, frame2 = cap.read() # Bir sonraki kareyi oku

        if not ret: # Eğer kare alınamadıysa döngüden çık
            print("Video sonuna ulaşıldı veya dosya okunamıyor.")
            break
                
        nextt = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY) # Yeni kareyi gri tonlamaya çevir
        
        # Lucas-Kanade optik akışının hesaplanması
        p1, st, err = cv.calcOpticalFlowPyrLK(prvs,         # Bir önceki gri kare.
                                              nextt,        # Şu anki gri kare.
                                              p0,           # Önceki karedeki izlenen özellik noktaları.
                                              None,         # Optik akışın hesaplanacağı sonraki noktalar.
                                              **lk_params)   # Lucas-Kanade parametreleri unpacking ile geçirilir.

        # Sadece iyi izlenilebilen noktaları seç
        good_new = p1[st == 1] # Yeni karedeki izlenebilen noktalar.
        good_old = p0[st == 1] # Önceki karedeki izlenebilen noktalar.

        # İyi izlenen noktalar arasında çizgiler çizilir ve noktalar işaretlenir.
        for i, (new, old) in enumerate(zip(good_new, good_old)): # Yeni ve eski noktalar sırasıyla işlenir.
            a, b = new.ravel() # Yeni nokta koordinatları (x, y).
            c, d = old.ravel() # Eski nokta koordinatları (x, y).
            mask    = cv.line(mask,             # Maske üzerinde bir çizgi çizilir.
                             (int(a), int(b)),  # Yeni nokta koordinatları.
                             (int(c), int(d)),  # Eski nokta koordinatları.
                             color[i].tolist(), # Rastgele renk kullanılır.
                             2)                 # Çizgi kalınlığı.
            
            mask    = cv.circle(mask,             # Yeni noktanın üzerine daire çizilir.
                               (int(a), int(b)),  # Yeni noktanın koordinatları.
                               2,                 # Dairenin yarıçapı.
                               color[i].tolist(), # Rastgele renk kullanılır.
                               -1)                # Daire dolu çizilir.
        
        # Maske görüntüsünü orijinal video karesi ile birleştir.
        img = cv.add(frame2, mask) # Orijinal kare ile optik akış maskesi birleştirilir.

        cv.imshow('Original Video', frame2) # Orijinal video karesi gösterilir.
        cv.imshow('Sparse Optical Flow - Lucas-Kanade', img) # Optik akış görüntüsü gösterilir.
        
        # 'Esc' tuşuna basarak döngüyü sonlandır
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

        prvs = nextt.copy() # Sonraki kareyi bir önceki kare olarak ayarla
        p0   = good_new.reshape(-1,1,2)

In [275]:
lucas_kanade_opt_flow(prvs)

Video sonuna ulaşıldı veya dosya okunamıyor.


In [276]:
cap.release()
cv.destroyAllWindows() # Tüm pencereleri kapat

![](ss_of_videos_4.png)

In [277]:
![](ss_of_videos_5.png)

'[]' is not recognized as an internal or external command,
operable program or batch file.


mark down ile fotoğraf eklemeye çalışıyorum ama 2. fotoğraf eklenmiyor

Hücre 1 (çalışıyor)
![](ss_of_videos_4.png)
Hücre 2 (çalışmıyor)
![](ss_of_videos_5.png)
'[]' is not recognized as an internal or external command,
operable program or batch file.